In [1]:
#import sys
#!{sys.executable} -m pip install --upgrade "scikit-learn>=1.6.1,<1.7" category_encoders shap lyft_fugue

In [1]:
#usual packages
import sklearn
import numpy as np
from IPython.display import IFrame
from sklearn.model_selection import GroupKFold
from datetime import date
import pandas as pd
from pandas import DataFrame
import subprocess
import sys

#lyft packages
import lyft_data_toolkit.db.write.df
from lyft_data_toolkit.db.read import Compute
from lyft_data_toolkit.db.read import Query
from lyft_data_toolkit.db.read.df import to_dataframe  
from lyft_data_toolkit.v1_to_v3 import hive
from lyft_data_toolkit.v1_to_v3 import presto
from lyft_data_toolkit.v1_to_v3 import spark
from lyft_data_toolkit.v1_to_v3 import trino

#other packages
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [2]:
#pull data (might be useless)
SQL ="""
--higherlimit
WITH pre_final_table AS (
SELECT DISTINCT 
       sub1.rider_lyft_id,
       sub1.business_program_user_id,
       sub1.business_program_type,
       sub1.business_program_type_cleaned,
       sub1.lyft_signed_up_at,
       sub1.bp_created_at,
       sub1.bp_activated_at,
       sub1.user_type,
       sub1.sub_user_type,
       sub2.first_ride_finished_at
  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1
  LEFT JOIN coco.dim_user sub2 
    ON sub1.rider_lyft_id = sub2.user_lyft_id
 WHERE sub1.is_business_ride = TRUE 
) 

,pre_final_table_v2 AS (
SELECT rider_lyft_id,
       business_program_user_id,
       business_program_type,
       business_program_type_cleaned,
       lyft_signed_up_at,
       bp_created_at,
       bp_activated_at,
       user_type,
       sub_user_type,
       first_ride_finished_at,
       ROW_NUMBER() OVER(PARTITION BY rider_lyft_id ORDER BY bp_activated_at DESC) rnk 
  FROM pre_final_table 
 WHERE TRUE
)

SELECT DATE_TRUNC('week',CAST(bp_activated_at AS DATE)) week,
       business_program_type_cleaned,
       CASE WHEN DATE_DIFF('day',first_ride_finished_at,bp_activated_at) > 0 THEN 'PP' ELSE 'NP' END AS activation_type, -- proxy for NP/PP
       COUNT(DISTINCT rider_lyft_id) count_activations
  FROM pre_final_table_v2
 WHERE TRUE 
   AND rnk = 1
 GROUP BY 1,2,3
"""  
df = pd.DataFrame()
df = presto.query(SQL, scheduled=True)

{"ts": "2025-11-13T21:19:18.456Z", "uuid": "7e6a283d-bde7-449e-a82a-14118454cd41", "name": "lyft_data_toolkit.v1_to_v3.presto", "lvlname": "WARNING", "lvlno": 30, "pid": 4059, "call_site": "code/venvs/venv/lib/python3.10/site-packages/lyft_data_toolkit/v1_to_v3/presto.py:18", "revision": "b8758af9c9", "msg": "DEPRECATION WARNING: presto.query method is outdated.Please use the new query function in the lyft_data_toolkit package."}
/code/venvs/venv/lib/python3.10/site-packages/lyft_stats/client.py:347: UserWarning: Attempting to establish a new TCP stats connection
  warnings.warn("Attempting to establish a new TCP stats connection")


Created Mozart command
{'mozart_id': 567399756, 'backend': 'presto', 'native_command_id': '', 'job_url': 'https://sql.lyft.net/databases/mozart/queries/567399756', 'query': "--higherlimit\nWITH pre_final_table AS (\nSELECT DISTINCT \n       sub1.rider_lyft_id,\n       sub1.business_program_user_id,\n       sub1.business_program_type,\n       sub1.business_program_type_cleaned,\n       sub1.lyft_signed_up_at,\n       sub1.bp_created_at,\n       sub1.bp_activated_at,\n       sub1.user_type,\n       sub1.sub_user_type,\n       sub2.first_ride_finished_at\n  FROM shuninga.causal_analysis_bp_raw_data_step_4 sub1\n  LEFT JOIN coco.dim_user sub2 \n    ON sub1.rider_lyft_id = sub2.user_lyft_id\n WHERE sub1.is_business_ride = TRUE \n) \n\n,pre_final_table_v2 AS (\nSELECT rider_lyft_id,\n       business_program_user_id,\n       business_program_type,\n       business_program_type_cleaned,\n       lyft_signed_up_at,\n       bp_created_at,\n       bp_activated_at,\n       user_type,\n       sub_us

In [3]:
df.columns

Index(['week', 'business_program_type_cleaned', 'activation_type',
       'count_activations'],
      dtype='object')

In [4]:
#pivot table to have columns for managed vs organic activations
df_pivot = df.pivot_table(
    index=["week", "activation_type"],
    columns="business_program_type_cleaned",
    values="count_activations",
    fill_value=0
).reset_index()

print(df_pivot)

business_program_type_cleaned       week activation_type  managed  organic
0                             2021-08-09              NP       10        1
1                             2021-08-09              PP      932      850
2                             2021-08-16              NP       39        4
3                             2021-08-16              PP     1962     1720
4                             2021-08-23              NP       16        3
..                                   ...             ...      ...      ...
441                           2025-10-27              PP     1417     3990
442                           2025-11-03              NP      187      141
443                           2025-11-03              PP     1479     4608
444                           2025-11-10              NP       50       33
445                           2025-11-10              PP      516     1453

[446 rows x 4 columns]


In [5]:
np_df = df_pivot[df_pivot.activation_type == "NP"]
pp_df = df_pivot[df_pivot.activation_type == "PP"]

In [6]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from joblib import load

launch_date = "2025-08-04"

np_df = df_pivot[df_pivot["activation_type"] == "NP"].copy()
pp_df = df_pivot[df_pivot["activation_type"] == "PP"].copy()

def add_lags(df):
    df = df.sort_values("week").reset_index(drop=True)
    df["organic_lag1"] = df["organic"].shift(1).fillna(method='bfill')
    df["organic_lag2"] = df["organic"].shift(2).fillna(method='bfill')
    return df

np_df = add_lags(np_df)
pp_df = add_lags(pp_df)

def predict_post_rewards(df, model_file, use_lag2=True):
    post_df = df[df["week"] >= launch_date].copy()
    if post_df.empty:
        print(f"No post-rewards data for {model_file}")
        return None
    
    #features = ["organic", "organic_lag1"]
    #if use_lag2:
        #features.append("organic_lag2")
    
    X_post = post_df["organic"]
    
    model = load(model_file)
    
    post_df["predicted_managed"] = model.predict(
        start=len(df[df["week"] < launch_date]),
        end=len(df)-1,
        exog=X_post
    )
    
    return post_df[["week", "managed", "predicted_managed"]]

np_post_preds = predict_post_rewards(np_df, "arimax_model_np.pkl", use_lag2=False)
pp_post_preds = predict_post_rewards(pp_df, "arimax_model_pp.pkl", use_lag2=True)

if np_post_preds is not None:
    np_post_preds.to_csv("np_post_rewards_predictions.csv", index=False)
    print("NP post-rewards predictions exported to np_post_rewards_predictions.csv")

if pp_post_preds is not None:
    pp_post_preds.to_csv("pp_post_rewards_predictions.csv", index=False)
    print("PP post-rewards predictions exported to pp_post_rewards_predictions.csv")

print("NP Post-Rewards Predictions:")
print(np_post_preds)

print("\nPP Post-Rewards Predictions:")
print(pp_post_preds)


NP post-rewards predictions exported to np_post_rewards_predictions.csv
PP post-rewards predictions exported to pp_post_rewards_predictions.csv
NP Post-Rewards Predictions:
business_program_type_cleaned       week  managed  predicted_managed
208                           2025-08-04      134         133.393125
209                           2025-08-11      120         142.826053
210                           2025-08-18      131         141.851859
211                           2025-08-25       92         121.021250
212                           2025-09-01      109         110.594195
213                           2025-09-08      153         160.600157
214                           2025-09-15      172         162.041984
215                           2025-09-22      151         143.659792
216                           2025-09-29      170         126.124932
217                           2025-10-06      195         151.968766
218                           2025-10-13      146         167.915443